#Ejercicio 2: Image Mirroring

##1. Introducción
<justify>En este cuaderno, vamos a realizar el espejado de una imagen de manera horizontal [2]. Este tratamiento de la imagen se hará de manera secuencial y en paralelo optimizada con CUDA [4].

El algoritmo que se desarrollará a continuación está basado en la transposición de matrices [1]. La diferencia está en que solamente se hará sobre el eje x, a modo de que quede con el efecto de espejado horizontal y no realmente transpuesta completa.

Matemáticamente hablando, se procederá a realizar la siguiente operación:
</justify>

<center>$imagen[x_i][y]= imagen[n -1 - x_i][y]$    [3]</center>

<justify>Siendo *imagen* la imagen en forma de matriz, $x_i$ va a ser la posición del eje x que se va a trasponer, *n* es la cantidad total de elementos sobre el eje x (es decir, el ancho). Nótese que *y* queda fijo en toda la operación, ya que los valores sobre el eje y se mantienen debido a que es una transposición solamente sobre el eje x.
</justify>

##2. Armado del ambiente
Para el armado del ambiente, es necesario instalar CUDA [5] e importar una imagen a elección.

###2.1. Instalación de CUDA
Procedemos a instalar CUDA en el notebook con el siguiente comando:

In [ ]:
!pip install pycuda

###2.2. Importar imagen
Solicitamos al usuario que ingrese la URL de una imagen a elección. El formato de la imagen tiene que ser **.jpg** o **.jpeg** [2].

In [ ]:
#----- Usamos esta parte para que el usuario ingrese los parámetros -----
#@title Ingrese la dirección URL de una imagen .jpg / .jpeg { vertical-output: true }

imagen_url = "https://upload.wikimedia.org/wikipedia/commons/c/c3/NGC_4414_%28NASA-med%29.jpg" #@param {type: "string"}


"""
Se dejan algunas imágenes de prueba:

Cuadrada: https://miro.medium.com/max/400/0*C9qMhSqy-GdqP5aM.jpg
Más ancha: https://www.mundoperros.es/wp-content/uploads/2019/03/pitbull-terrier-feliz.jpg
Más larga: https://i.pinimg.com/originals/ab/80/70/ab80707d85d8ef6d5e6ee81da05c30c7.jpg
Súper grande: https://upload.wikimedia.org/wikipedia/commons/3/30/Amazona_aestiva_-upper_body-8a_%281%29.jpg
"""

if not (type(imagen_url) is str) or (imagen_url == ""):
  raise TypeError("Se tiene que ingresar una dirección URL")

if not (imagen_url.endswith(".jpg") or imagen_url.endswith(".jpeg")):
  raise Exception("La imagen tiene que ser en formato .jpg o .jpeg")

#Verificamos el parametro y tomamos la imagen
!wget {imagen_url} -O imagen.jpg

##3. Desarrollo
A continuación, se desarrollará el algoritmo para el espejado horizontal de la imagen del usuario.

In [ ]:
#Importamos los módulos necesarios
from datetime import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
%matplotlib inline


#CPU: empezamos a tomar el tiempo para el procesamiento
tiempo_total = dt.now()

#Definimos la función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#CPU: Abrimos la imagen
imagen_nombre = 'imagen.jpg'
imagen = Image.open(imagen_nombre)

#CPU: Tomamos las dimensiones de la imagen
img_ancho, img_alto = imagen.size

#CPU: convertimos la imagen en un array
img_original_cpu = np.asarray(imagen)
img_espejada_cpu = np.empty_like(img_original_cpu)
#Este lo tenemos para copiar el resultado de GPU
res_espejada_cpu = np.empty_like(img_original_cpu)

#GPU: reservamos la memoria GPU
img_original_gpu = cuda.mem_alloc(img_original_cpu.nbytes)
img_espejada_gpu = cuda.mem_alloc(img_espejada_cpu.nbytes)

#Copiamos la memoria a GPU
cuda.memcpy_htod(img_original_gpu, img_original_cpu)
cuda.memcpy_htod(img_espejada_gpu, img_espejada_cpu)

#CPU: definimos la función kernel que va a ejecutar GPU
module = SourceModule("""
__global__ void kernel_mirror(int alto, int ancho, char *img_original, char *img_espejada)
{
  //Calculamos las coordenadas del thread en 2D
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  int idy = threadIdx.y + blockIdx.y * blockDim.y;
  
  //Verificamos que los threads esten dentro de las dimensiones de la imagen
  if(idx < ancho && idy < alto)
  {
    //Creamos los nuevos indices necesarios para espejar la matriz
    int mirrorCol = ancho - idx - 1;
    int index_x = idy * ancho * 3 + idx * 3;
    int mirrorIdx = idy * ancho * 3 + mirrorCol * 3;
    
    //Tomamos cada uno de los 3 componentes RGB del pixel y lo copiamos
    img_espejada[mirrorIdx] = img_original[index_x];         //R
    img_espejada[mirrorIdx + 1] = img_original[index_x + 1]; //G
    img_espejada[mirrorIdx + 2] = img_original[index_x + 2]; //B
  }
}
""")

#CPU: mostramos los atributos de la imagen
print("Imagen: ", imagen_nombre, " - ", imagen.mode, " - [", str(img_ancho), ", ", str(img_alto), "]")

#CPU: generar la función kernel
kernel = module.get_function("kernel_mirror")


#CPU: se define el tamaño de las dimensiones
dim_hilo_x = 32
dim_bloque_x = np.int((img_ancho + dim_hilo_x - 1) / dim_hilo_x)

dim_hilo_y = 32
dim_bloque_y = np.int((img_alto + dim_hilo_y - 1) / dim_hilo_y)

print("Thread: [x: ", dim_hilo_x, ", y: ", dim_hilo_y, "] | Bloque: [x: ", dim_bloque_x, ", y: ", dim_bloque_y, "]")
print("Total de thread: [", dim_hilo_x * dim_bloque_x,
      ", ", dim_hilo_y * dim_bloque_y, "]",
      " = ", dim_hilo_x * dim_bloque_x * dim_hilo_y * dim_bloque_y)

#Comenzamos a tomar el tiempo de ejecución de GPU
tiempo_gpu = dt.now()

#GPU: Mandamos la función kernel
kernel(np.int32(img_alto), np.int32(img_ancho), img_original_gpu, img_espejada_gpu, block=(dim_hilo_x, dim_hilo_y, 1), grid=(dim_bloque_x, dim_bloque_y, 1))

#Cortamos el tiempo de GPU
tiempo_gpu = dt.now() - tiempo_gpu

#CPU: copia el resultado desde la memoria de GPU
cuda.memcpy_dtoh(res_espejada_cpu, img_espejada_gpu)

#CPU: realizamos versión secuencial y empezamos a tomar el tiempo de ejecución del bucle
tiempo_bucle_cpu = dt.now()

for idy in range(0, img_alto):
  for idx in range(0, img_ancho):
      #Trasponemos solamente el eje x, dejamos igual al eje y
      img_espejada_cpu[idy][idx] = img_original_cpu[idy][img_ancho - 1 - idx]


#CPU: Cortamos el tiempo del bucle de CPU
tiempo_bucle_cpu = dt.now() - tiempo_bucle_cpu

#Mostramos las imagenes resultantes
#Imagen original
plt.figure()
imgplot = plt.imshow(imagen)
#Resultado de CPU
plt.figure()
imgplot = plt.imshow(img_espejada_cpu)
#Resultado de GPU
plt.figure()
imgplot = plt.imshow(res_espejada_cpu)

#CPU: Tomamos el tiempo de ejecución total
tiempo_total = dt.now() - tiempo_total


#CPU: Informamos los tiempos de ejecución
print("Tiempo total: ", tiempo_en_ms(tiempo_total), "ms")
print("Tiempo del bucle en CPU: ", tiempo_en_ms(tiempo_bucle_cpu), "ms")
print("Tiempo de GPU: ", tiempo_en_ms(tiempo_gpu), "ms")

##4. Tabla de pasos
A continuación, realizaremos la tabla de pasos de ejecución del programa.

<center>

Procesador | Función | Detalle
--- | --- | ---
CPU | !pip install pycuda | Instalación de CUDA en el notebook actual
CPU | @param | Lectura del la dirección URL de la imagen desde el formulario de Colab
CPU | if... raise... | Controla que el parámetro pasado por el usuario sea válido
CPU | wget imagen_url | Lectura de la imagen a procesar desde la dirección URL proporcionada
CPU | import | Importa los módulos necesarios para la ejecución del programa
CPU | matplotlib inline | Macro necesaria para mostrar las imágenes
CPU | dt.now() | Toma el tiempo actual a modo de tiempo inicial de ejecución
CPU | Image.open() | Abre el archivo de la imagen
CPU | np.asarray(imagen) | Pasa la imagen al formato RAW
CPU | np.empty_like() | Generación de los arrays necesarios para el resultado de las imagenes
**GPU** | cuda.mem_alloc() | Reserva de la memoria para las imagenes en GPU
**GPU** | cuda.memcpy_htod() | Copia el la imagen *img_original_cpu* a la memoria de GPU
CPU | SourceModule() | Contiene el código del kernel
CPU | module.get_function() | Convierte el texto del kernel en una función de Python
CPU | dim_hilo_x, dim_hilo_y, dim_bloque_x, dim_bloque_y | Se calculan las dimensiones para la ejecución de 2D
**GPU** | kernel() | Ejecución del kernel en GPU junto con los parámetros necesarios
CPU | cuda.memcpy_dtoh() | Se copia desde la memoria GPU al CPU el resultado
CPU | loop for | Aplica el algoritmo secuencial a la imagen *img_reflejada_cpu*
CPU | plt.imshow() | Muestra las 3 imágenes en el orden: original, CPU y GPU
CPU | print() | Informa los resultados por pantalla

</center>

##5. Conclusiones
Podemos notar que el mayor tiempo de ejecución lo tiene la ejecución en el ciclo *for* de CPU, en el cual se realiza el espejado de la imagen en cuestión.

Esto es de esperarse, ya que es donde está el núcleo del programa que requiere más procesamiento. Cuanto más grandes sean las imágenes, más tiempo de ejecución va a requerir el ciclo *for*, ya que recorre mínimamente todas las posiciones de manera secuencial.

Notamos que GPU mantiene los mismos tiempos para cualquier tamaño de imagen. Podemos ver que la cantidad de hilos que se crean es muy cercana al tamaño de la matriz, con lo que podríamos casi estimar que cada pixel está siendo procesado por un hilo en paralelo.

##6. Bibliografía

* [1]: [Nvidia - Transposición de matrices usando CUDA](https://developer.nvidia.com/blog/efficient-matrix-transpose-cuda-cc/)
* [2]: [Introducción a la Programación en CUDA](https://riubu.ubu.es/bitstream/handle/10259/3933/Programacion_en_CUDA.pdf;jsessionid=FC6394584537780DD2AD3F995FD075B4?sequence=1)
* [3]: [Colab - Guía de markdowns](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=tPqPXAKKkzaM)
* [4]: [Página oficial de CUDA para Python](https://pypi.org/project/pycuda/)
* [5]: [Documentación de CUDA](https://documen.tician.de/pycuda/index.html)
